In [2]:
import torch
import base64
import struct
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import json
import csv
import seaborn as sns
from torch.autograd import Variable

# Funções de preparação e treinamento

In [3]:
# Retorna a(s) posição(ões) como valor(es) inteiro(s) a partir da array de predição
def get_position_from_markers_prediction(prediction):
    array = np.array(prediction)
    indices = np.where(array == 1)
    return list(indices[0])

In [4]:
# Abre o dataset de um marcador específico e retorna o dataframe, features e rótulos de 10.000 dados
def load_dataset(marker):
    csv_name = 'only_' + marker + '_label.csv'
    df = pd.read_csv(csv_name)
    df = df.head(10000)

    X = df.iloc[:,0:430]
    y = df.iloc[:,-430:]
    return df, X, y

In [5]:
# Separa as features e rótulos em conjuntos de treino, validação e teste
def get_tensors(X, y):
    train_size = math.floor(X.shape[0] * 0.6)
    valid_size = math.floor(X.shape[0] * 0.8)
    
    X_train_set = X[:train_size].to_numpy()
    X_valid_set = X[train_size:valid_size].to_numpy()
    X_test_set = X[valid_size:].to_numpy()
    y_train_set = y[:train_size].to_numpy()
    y_valid_set = y[train_size:valid_size].to_numpy()
    y_test_set = y[valid_size:].to_numpy()

    X_train_tensors = Variable(torch.Tensor(X_train_set))
    X_valid_tensors = Variable(torch.Tensor(X_valid_set))
    X_test_tensors = Variable(torch.Tensor(X_test_set))

    y_train_tensors = Variable(torch.Tensor(y_train_set))
    y_valid_tensors = Variable(torch.Tensor(y_valid_set))
    y_test_tensors = Variable(torch.Tensor(y_test_set))
    
    X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
    X_valid_tensors_final = torch.reshape(X_valid_tensors,  (X_valid_tensors.shape[0], 1, X_valid_tensors.shape[1]))
    X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1]))
    return X_train_tensors_final.cuda(), y_train_tensors.cuda(), X_valid_tensors_final.cuda(), y_valid_tensors.cuda(), X_test_tensors_final.cuda(), y_test_tensors.cuda()

In [6]:
# Arquitetura do modelo de aprendizado profundo
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True, dropout=0, bidirectional=True, device=device)
        self.fc_1 =  nn.Linear(hidden_size, 256, device=device)
        self.fc = nn.Linear(256, num_classes, device=device)

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size))
        h_0 = h_0.cuda()
        c_0 = Variable(torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size))
        c_0 = c_0.cuda()
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn[self.num_layers - 1]
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        return out

In [7]:
# Função de treinamento do modelo
def train_lstm(
    X_train_tensors, 
    y_train_tensors,
    X_valid,
    y_valid,
    marker,
    num_epochs = 1000,
    learning_rate = 0.01,
    input_size = 430,
    hidden_size = 100,
    num_layers = 1, 
    num_classes = 430
):

    best_lstm1 = None
    lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1]) #our lstm class 
    criterion = torch.nn.CrossEntropyLoss()
    #criterion = torch.nn.BCEWithLogitsLoss() 
    optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)
    best_acc_3 = 0
    for epoch in range(1, num_epochs + 1):
        outputs = lstm1.forward(X_train_tensors)
        optimizer.zero_grad()

        loss = criterion(outputs, y_train_tensors)

        loss.backward()

        optimizer.step() 
       
        correct, correct_per_one, correct_per_two, correct_per_three, totally_wrong, total = 0, 0, 0, 0, 0, 0
        for i in range(0, X_valid.size(0)):
            x_val, y_val = torch.unsqueeze(X_valid[i], 0), y_valid[i]
            out = lstm1(x_val)
            preds = torch.topk(out.flatten(), 1).indices.item()
            total += 1
            correct_value = torch.topk(y_val.flatten(), 1).indices.item()
            
            # medidores de acurácia por diferentes margens
            correct += (preds == correct_value)
            correct_per_one += (correct_value <= preds +1 and correct_value >= preds -1)
            correct_per_two += (correct_value <= preds +2 and correct_value >= preds -2)
            correct_per_three += (correct_value <= preds +3 and correct_value >= preds -3)
            totally_wrong += (correct_value >= preds +10 or correct_value <= -10)
            
            acc = correct / total
            acc_1 = correct_per_one / total
            acc_2 = correct_per_two / total
            acc_3 = correct_per_three / total
            wrong = totally_wrong/total

            
        # Caso encontre a melhora acurácia, salva o estado do modelo
        if acc_3 > best_acc_3:
            best_acc_3 = acc_3
            torch.save(lstm1.state_dict(), 'models/best_' + marker)
            best_lstm1 = lstm1
            best_wrong = wrong
        if (epoch % 5 == 0):
            print("Marker: %s, Epoch: %d, loss: %1.5f, Acc: %f, Acc1: %f, Acc2: %f, Acc3: %f, Wrong: %f" % (marker, epoch, loss.item(), round(acc * 100, 2), round(acc_1 * 100, 2), round(acc_2 * 100, 2), round(acc_3 * 100, 2), round(wrong * 100, 2)))
    print("Marker %s best Accuracy by 3 margin: %.2f, best wrong: %.2f" % (marker, best_acc_3 * 100, best_wrong * 100))
    return best_lstm1

# Treino dos modelos

In [9]:
marker = 'P_INI'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: P_INI, Epoch: 5, loss: 4.45992, Acc: 3.550000, Acc1: 10.050000, Acc2: 16.900000, Acc3: 24.150000, Wrong: 12.850000
Marker: P_INI, Epoch: 10, loss: 3.99309, Acc: 4.150000, Acc1: 13.300000, Acc2: 20.400000, Acc3: 28.100000, Wrong: 14.150000
Marker: P_INI, Epoch: 15, loss: 3.57922, Acc: 5.450000, Acc1: 16.950000, Acc2: 28.950000, Acc3: 38.350000, Wrong: 6.650000
Marker: P_INI, Epoch: 20, loss: 2.97753, Acc: 10.700000, Acc1: 30.200000, Acc2: 48.700000, Acc3: 61.550000, Wrong: 3.650000
Marker: P_INI, Epoch: 25, loss: 2.63464, Acc: 14.500000, Acc1: 40.150000, Acc2: 60.850000, Acc3: 74.250000, Wrong: 2.700000
Marker: P_INI, Epoch: 30, loss: 2.45614, Acc: 16.900000, Acc1: 45.950000, Acc2: 66.050000, Acc3: 77.450000, Wrong: 2.050000
Marker: P_INI, Epoch: 35, loss: 2.32899, Acc: 17.450000, Acc1: 48.300000, Acc2: 68.300000, Acc3: 78.900000, Wrong: 1.550000
Marker: P_INI, Epoch: 40, loss: 2.21220, Acc: 19.700000, Acc1: 50.600000, Acc2: 70.150000, Acc3: 80.100000, Wrong: 1.400000
Marker: P_

In [10]:
marker = 'P'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: P, Epoch: 5, loss: 4.53606, Acc: 4.300000, Acc1: 12.350000, Acc2: 20.500000, Acc3: 27.750000, Wrong: 12.100000
Marker: P, Epoch: 10, loss: 3.96845, Acc: 5.500000, Acc1: 14.350000, Acc2: 22.350000, Acc3: 32.700000, Wrong: 9.900000
Marker: P, Epoch: 15, loss: 3.48644, Acc: 6.850000, Acc1: 20.200000, Acc2: 33.600000, Acc3: 45.250000, Wrong: 5.900000
Marker: P, Epoch: 20, loss: 2.84103, Acc: 17.450000, Acc1: 47.600000, Acc2: 66.250000, Acc3: 77.200000, Wrong: 2.700000
Marker: P, Epoch: 25, loss: 2.31855, Acc: 27.400000, Acc1: 66.600000, Acc2: 82.050000, Acc3: 89.150000, Wrong: 2.400000
Marker: P, Epoch: 30, loss: 2.02611, Acc: 32.450000, Acc1: 73.950000, Acc2: 86.400000, Acc3: 91.100000, Wrong: 2.200000
Marker: P, Epoch: 35, loss: 1.80218, Acc: 38.050000, Acc1: 80.550000, Acc2: 89.550000, Acc3: 92.450000, Wrong: 2.250000
Marker: P, Epoch: 40, loss: 1.60493, Acc: 43.050000, Acc1: 80.950000, Acc2: 90.250000, Acc3: 92.650000, Wrong: 2.350000
Marker: P, Epoch: 45, loss: 1.43441, Acc: 4

In [12]:
marker = 'P_END'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: P_END, Epoch: 5, loss: 4.50432, Acc: 4.100000, Acc1: 12.100000, Acc2: 18.600000, Acc3: 25.900000, Wrong: 5.800000
Marker: P_END, Epoch: 10, loss: 3.92676, Acc: 4.200000, Acc1: 12.200000, Acc2: 20.900000, Acc3: 29.400000, Wrong: 16.950000
Marker: P_END, Epoch: 15, loss: 3.47509, Acc: 6.150000, Acc1: 19.250000, Acc2: 30.750000, Acc3: 42.550000, Wrong: 5.550000
Marker: P_END, Epoch: 20, loss: 2.90606, Acc: 11.450000, Acc1: 30.850000, Acc2: 49.400000, Acc3: 63.950000, Wrong: 3.500000
Marker: P_END, Epoch: 25, loss: 2.52676, Acc: 15.850000, Acc1: 44.650000, Acc2: 65.750000, Acc3: 79.850000, Wrong: 2.100000
Marker: P_END, Epoch: 30, loss: 2.31582, Acc: 18.500000, Acc1: 50.600000, Acc2: 73.550000, Acc3: 85.650000, Wrong: 1.700000
Marker: P_END, Epoch: 35, loss: 2.15620, Acc: 21.000000, Acc1: 56.300000, Acc2: 76.800000, Acc3: 87.400000, Wrong: 1.400000
Marker: P_END, Epoch: 40, loss: 2.03591, Acc: 22.850000, Acc1: 58.700000, Acc2: 78.700000, Acc3: 88.500000, Wrong: 1.250000
Marker: P_E

In [15]:
marker = 'QRS_INI'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: QRS_INI, Epoch: 5, loss: 4.10582, Acc: 8.300000, Acc1: 23.850000, Acc2: 36.400000, Acc3: 48.000000, Wrong: 5.000000
Marker: QRS_INI, Epoch: 10, loss: 3.12382, Acc: 11.100000, Acc1: 33.150000, Acc2: 51.450000, Acc3: 67.250000, Wrong: 3.600000
Marker: QRS_INI, Epoch: 15, loss: 2.55740, Acc: 16.050000, Acc1: 44.400000, Acc2: 65.200000, Acc3: 78.700000, Wrong: 2.550000
Marker: QRS_INI, Epoch: 20, loss: 2.39666, Acc: 16.400000, Acc1: 49.300000, Acc2: 71.050000, Acc3: 83.500000, Wrong: 1.900000
Marker: QRS_INI, Epoch: 25, loss: 2.25806, Acc: 19.650000, Acc1: 50.650000, Acc2: 73.650000, Acc3: 84.400000, Wrong: 1.750000
Marker: QRS_INI, Epoch: 30, loss: 2.14191, Acc: 20.900000, Acc1: 54.500000, Acc2: 76.350000, Acc3: 86.200000, Wrong: 1.450000
Marker: QRS_INI, Epoch: 35, loss: 2.01602, Acc: 24.950000, Acc1: 58.350000, Acc2: 77.300000, Acc3: 87.300000, Wrong: 1.450000
Marker: QRS_INI, Epoch: 40, loss: 1.88578, Acc: 26.000000, Acc1: 62.550000, Acc2: 79.750000, Acc3: 87.600000, Wrong: 1.3

In [16]:
marker = 'R'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: R, Epoch: 5, loss: 3.85009, Acc: 9.550000, Acc1: 27.300000, Acc2: 42.850000, Acc3: 58.800000, Wrong: 4.950000
Marker: R, Epoch: 10, loss: 2.81791, Acc: 21.150000, Acc1: 58.400000, Acc2: 79.800000, Acc3: 86.950000, Wrong: 3.700000
Marker: R, Epoch: 15, loss: 1.96761, Acc: 41.350000, Acc1: 81.800000, Acc2: 89.200000, Acc3: 90.450000, Wrong: 3.900000
Marker: R, Epoch: 20, loss: 1.62099, Acc: 54.050000, Acc1: 88.700000, Acc2: 91.700000, Acc3: 92.700000, Wrong: 3.050000
Marker: R, Epoch: 25, loss: 1.30320, Acc: 61.850000, Acc1: 89.950000, Acc2: 92.500000, Acc3: 93.200000, Wrong: 2.600000
Marker: R, Epoch: 30, loss: 1.06582, Acc: 69.150000, Acc1: 91.350000, Acc2: 92.950000, Acc3: 93.350000, Wrong: 2.350000
Marker: R, Epoch: 35, loss: 0.85520, Acc: 74.400000, Acc1: 92.000000, Acc2: 93.700000, Acc3: 94.050000, Wrong: 2.000000
Marker: R, Epoch: 40, loss: 0.67713, Acc: 77.600000, Acc1: 92.600000, Acc2: 94.300000, Acc3: 94.700000, Wrong: 1.250000
Marker: R, Epoch: 45, loss: 0.54343, Acc: 

In [17]:
marker = 'S'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: S, Epoch: 5, loss: 4.20057, Acc: 4.200000, Acc1: 13.600000, Acc2: 22.250000, Acc3: 31.650000, Wrong: 16.300000
Marker: S, Epoch: 10, loss: 3.30046, Acc: 11.950000, Acc1: 32.900000, Acc2: 51.250000, Acc3: 65.150000, Wrong: 5.000000
Marker: S, Epoch: 15, loss: 2.76275, Acc: 16.700000, Acc1: 45.050000, Acc2: 64.850000, Acc3: 76.900000, Wrong: 5.050000
Marker: S, Epoch: 20, loss: 2.53774, Acc: 21.800000, Acc1: 56.400000, Acc2: 74.500000, Acc3: 81.350000, Wrong: 5.050000
Marker: S, Epoch: 25, loss: 2.29893, Acc: 25.300000, Acc1: 62.250000, Acc2: 77.450000, Acc3: 83.100000, Wrong: 5.100000
Marker: S, Epoch: 30, loss: 2.07314, Acc: 30.300000, Acc1: 67.700000, Acc2: 79.800000, Acc3: 85.250000, Wrong: 4.650000
Marker: S, Epoch: 35, loss: 1.86524, Acc: 34.300000, Acc1: 71.000000, Acc2: 82.500000, Acc3: 86.750000, Wrong: 4.000000
Marker: S, Epoch: 40, loss: 1.67200, Acc: 39.000000, Acc1: 73.250000, Acc2: 83.450000, Acc3: 87.350000, Wrong: 3.350000
Marker: S, Epoch: 45, loss: 1.55424, Acc:

In [18]:
marker = 'QRS_END'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: QRS_END, Epoch: 5, loss: 4.17847, Acc: 5.050000, Acc1: 16.150000, Acc2: 26.450000, Acc3: 36.700000, Wrong: 9.300000
Marker: QRS_END, Epoch: 10, loss: 3.26787, Acc: 9.850000, Acc1: 29.950000, Acc2: 46.400000, Acc3: 59.350000, Wrong: 5.900000
Marker: QRS_END, Epoch: 15, loss: 2.85782, Acc: 11.200000, Acc1: 35.550000, Acc2: 54.450000, Acc3: 68.000000, Wrong: 4.500000
Marker: QRS_END, Epoch: 20, loss: 2.67661, Acc: 13.900000, Acc1: 39.650000, Acc2: 58.650000, Acc3: 72.300000, Wrong: 4.050000
Marker: QRS_END, Epoch: 25, loss: 2.54283, Acc: 15.100000, Acc1: 40.950000, Acc2: 61.400000, Acc3: 73.150000, Wrong: 3.850000
Marker: QRS_END, Epoch: 30, loss: 2.42387, Acc: 16.050000, Acc1: 45.700000, Acc2: 63.600000, Acc3: 76.300000, Wrong: 3.850000
Marker: QRS_END, Epoch: 35, loss: 2.29962, Acc: 18.250000, Acc1: 48.950000, Acc2: 66.300000, Acc3: 78.450000, Wrong: 3.200000
Marker: QRS_END, Epoch: 40, loss: 2.16546, Acc: 21.700000, Acc1: 51.600000, Acc2: 69.100000, Acc3: 79.850000, Wrong: 3.15

In [19]:
marker = 'T_INI'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: T_INI, Epoch: 5, loss: 4.70854, Acc: 3.150000, Acc1: 9.100000, Acc2: 14.400000, Acc3: 19.750000, Wrong: 16.550000
Marker: T_INI, Epoch: 10, loss: 4.28053, Acc: 3.800000, Acc1: 9.650000, Acc2: 15.450000, Acc3: 21.500000, Wrong: 42.100000
Marker: T_INI, Epoch: 15, loss: 3.98820, Acc: 3.650000, Acc1: 10.200000, Acc2: 16.100000, Acc3: 22.350000, Wrong: 23.500000
Marker: T_INI, Epoch: 20, loss: 3.88578, Acc: 4.150000, Acc1: 11.050000, Acc2: 17.050000, Acc3: 23.300000, Wrong: 18.700000
Marker: T_INI, Epoch: 25, loss: 3.77701, Acc: 3.500000, Acc1: 11.550000, Acc2: 18.000000, Acc3: 25.450000, Wrong: 25.650000
Marker: T_INI, Epoch: 30, loss: 3.66769, Acc: 3.900000, Acc1: 12.900000, Acc2: 20.050000, Acc3: 27.650000, Wrong: 20.100000
Marker: T_INI, Epoch: 35, loss: 3.55499, Acc: 4.200000, Acc1: 12.800000, Acc2: 21.300000, Acc3: 29.550000, Wrong: 17.600000
Marker: T_INI, Epoch: 40, loss: 3.44698, Acc: 4.700000, Acc1: 14.950000, Acc2: 23.300000, Acc3: 31.650000, Wrong: 17.400000
Marker: T_I

In [20]:
marker = 'T'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: T, Epoch: 5, loss: 4.71438, Acc: 2.750000, Acc1: 8.850000, Acc2: 15.050000, Acc3: 22.200000, Wrong: 12.750000
Marker: T, Epoch: 10, loss: 4.20001, Acc: 4.300000, Acc1: 10.850000, Acc2: 17.950000, Acc3: 23.900000, Wrong: 15.800000
Marker: T, Epoch: 15, loss: 3.93395, Acc: 3.350000, Acc1: 11.800000, Acc2: 20.100000, Acc3: 28.300000, Wrong: 17.550000
Marker: T, Epoch: 20, loss: 3.82581, Acc: 4.250000, Acc1: 13.850000, Acc2: 22.400000, Acc3: 29.500000, Wrong: 19.000000
Marker: T, Epoch: 25, loss: 3.62083, Acc: 7.100000, Acc1: 17.550000, Acc2: 27.450000, Acc3: 38.050000, Wrong: 11.850000
Marker: T, Epoch: 30, loss: 3.37244, Acc: 8.700000, Acc1: 24.200000, Acc2: 38.450000, Acc3: 48.000000, Wrong: 7.250000
Marker: T, Epoch: 35, loss: 3.17882, Acc: 10.050000, Acc1: 28.500000, Acc2: 41.950000, Acc3: 54.300000, Wrong: 9.550000
Marker: T, Epoch: 40, loss: 2.95705, Acc: 11.650000, Acc1: 33.450000, Acc2: 48.450000, Acc3: 58.550000, Wrong: 6.200000
Marker: T, Epoch: 45, loss: 2.75639, Acc: 1

In [21]:
marker = 'T_END'
df, X, y = load_dataset(marker)
X_train, y_train, X_val, y_val, X_test, y_test = get_tensors(X, y)
lstm = train_lstm(X_train, y_train, X_val, y_val, marker=marker, num_layers=3, hidden_size=400,num_epochs=100)

Marker: T_END, Epoch: 5, loss: 4.51355, Acc: 4.300000, Acc1: 11.400000, Acc2: 18.650000, Acc3: 25.600000, Wrong: 21.550000
Marker: T_END, Epoch: 10, loss: 4.12939, Acc: 3.350000, Acc1: 9.100000, Acc2: 15.850000, Acc3: 23.150000, Wrong: 16.000000
Marker: T_END, Epoch: 15, loss: 3.83706, Acc: 3.500000, Acc1: 12.950000, Acc2: 20.650000, Acc3: 27.750000, Wrong: 16.900000
Marker: T_END, Epoch: 20, loss: 3.71806, Acc: 4.600000, Acc1: 12.450000, Acc2: 20.500000, Acc3: 29.400000, Wrong: 23.050000
Marker: T_END, Epoch: 25, loss: 3.54785, Acc: 5.200000, Acc1: 16.050000, Acc2: 26.250000, Acc3: 35.850000, Wrong: 16.150000
Marker: T_END, Epoch: 30, loss: 3.38802, Acc: 5.450000, Acc1: 18.650000, Acc2: 31.000000, Acc3: 42.700000, Wrong: 11.900000
Marker: T_END, Epoch: 35, loss: 3.21319, Acc: 6.950000, Acc1: 21.500000, Acc2: 34.600000, Acc3: 46.450000, Wrong: 8.650000
Marker: T_END, Epoch: 40, loss: 3.05923, Acc: 7.800000, Acc1: 24.250000, Acc2: 39.650000, Acc3: 51.000000, Wrong: 9.400000
Marker: T_EN

# Teste e avaliação dos modelos criados

In [11]:
markers = ['P_INI', 'P', 'P_END', 'QRS_INI', 'R', 'S', 'QRS_END', 'T_INI', 'T', 'T_END']
lstms_dict = {
    'P_INI': None,
    'P': None,
    'P_END': None,
    'QRS_INI': None, 
    'R': None, 
    'S': None, 
    'QRS_END': None, 
    'T_INI': None, 
    'T': None, 
    'T_END': None
}

In [15]:
for marker in markers:
    path = 'models/best_' + marker
    model = LSTM1(num_classes=430, input_size=430, hidden_size=400, num_layers=3, seq_length=430)
    model.load_state_dict(torch.load(path))

    lstms_dict[marker] = model

In [16]:
marker = 'P_INI'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker P_INI Acc: 85.50 Wrong 4.20


In [28]:
marker = 'P'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker P Acc: 93.85 Wrong 4.00


In [29]:
marker = 'P_END'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker P_END Acc: 90.40 Wrong 2.85


In [30]:
marker = 'QRS_INI'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker QRS_INI Acc: 90.85 Wrong 2.25


In [31]:
marker = 'R'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker R Acc: 96.55 Wrong 1.85


In [32]:
marker = 'S'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker S Acc: 87.05 Wrong 4.75


In [36]:
marker = 'QRS_END'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker QRS_END Acc: 81.25 Wrong 4.50


In [33]:
marker = 'T_INI'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker T_INI Acc: 34.20 Wrong 29.00


In [34]:
marker = 'T'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker T Acc: 66.95 Wrong 20.50


In [35]:
marker = 'T_END'
df, X, y = load_dataset(marker)
X_train_tensors, y_train_tensors, X_val, y_val, X_test, y_test = get_tensors(X, y)
predictions = lstms_dict[marker](X_test)
correct = 0
wrong = 0
total = 0
for i in range(predictions.shape[0]):
    total += 1
    pred = predictions[i].cpu()
    pred = torch.topk(pred.flatten(), 1).indices.detach().numpy()[0]
    real = get_position_from_markers_prediction(y_test[i].cpu())[0]
    if pred >= real -3 and pred <= real + 3:
        correct += 1
    if pred >= real + 10 or pred <= real -10:
        wrong += 1

acc = (correct / total) * 100
err = (wrong / total) * 100
print('For marker %s Acc: %.2f Wrong %.2f' % (marker, acc, err))

For marker T_END Acc: 55.65 Wrong 14.45
